In [2]:
from mlflow import MlflowClient
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor

In [3]:
client = MlflowClient(tracking_uri="http://127.0.0.1:8080")
all_experiments = client.search_experiments()

In [4]:
default_experiment = [
    {"name": experiment.name, "lifecycle_stage": experiment.lifecycle_stage}
    for experiment in all_experiments
    if experiment.name == "Default"
][0]

In [5]:
pprint(default_experiment)

{'lifecycle_stage': 'active', 'name': 'Default'}


#### Creating the Apples Experiment with Meaningful tags

In [6]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "This is the grocery forecasting project. "
    "This experiment contains the produce models for apples."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "grocery-forecasting",
    "store_dept": "produce",
    "team": "stores-ml",
    "project_quarter": "Q3-2023",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
produce_apples_experiment = client.create_experiment(
    name="Apple_Models", tags=experiment_tags
)

RestException: RESOURCE_ALREADY_EXISTS: Experiment 'Apple_Models' already exists.

#### Searching based on tags

In [7]:
# Use search_experiments() to search on the project_name tag key

apples_experiment = client.search_experiments(
    filter_string="tags.`project_name` = 'grocery-forecasting'"
)

print(vars(apples_experiment[0]))

{'_experiment_id': '107281342746158191', '_name': 'Apple_Models', '_artifact_location': 'mlflow-artifacts:/107281342746158191', '_lifecycle_stage': 'active', '_tags': {'mlflow.experimentKind': 'custom_model_development', 'mlflow.note.content': 'This is the grocery forecasting project. This experiment contains the produce models for apples.', 'project_name': 'grocery-forecasting', 'project_quarter': 'Q3-2023', 'store_dept': 'produce', 'team': 'stores-ml'}, '_creation_time': 1758521512751, '_last_update_time': 1758521512751}


#### Defining a dataset generator

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


def generate_apple_sales_data_with_promo_adjustment(
    base_demand: int = 1000, n_rows: int = 5000
):
    """
    Generates a synthetic dataset for predicting apple sales demand with seasonality
    and inflation.

    This function creates a pandas DataFrame with features relevant to apple sales.
    The features include date, average_temperature, rainfall, weekend flag, holiday flag,
    promotional flag, price_per_kg, and the previous day's demand. The target variable,
    'demand', is generated based on a combination of these features with some added noise.

    Args:
        base_demand (int, optional): Base demand for apples. Defaults to 1000.
        n_rows (int, optional): Number of rows (days) of data to generate. Defaults to 5000.

    Returns:
        pd.DataFrame: DataFrame with features and target variable for apple sales prediction.

    Example:
        >>> df = generate_apple_sales_data_with_seasonality(base_demand=1200, n_rows=6000)
        >>> df.head()
    """

    # Set seed for reproducibility
    np.random.seed(9999)

    # Create date range
    dates = [datetime.now() - timedelta(days=i) for i in range(n_rows)]
    dates.reverse()

    # Generate features
    df = pd.DataFrame(
        {
            "date": dates,
            "average_temperature": np.random.uniform(10, 35, n_rows),
            "rainfall": np.random.exponential(5, n_rows),
            "weekend": [(date.weekday() >= 5) * 1 for date in dates],
            "holiday": np.random.choice([0, 1], n_rows, p=[0.97, 0.03]),
            "price_per_kg": np.random.uniform(0.5, 3, n_rows),
            "month": [date.month for date in dates],
        }
    )

    # Introduce inflation over time (years)
    df["inflation_multiplier"] = (
        1 + (df["date"].dt.year - df["date"].dt.year.min()) * 0.03
    )

    # Incorporate seasonality due to apple harvests
    df["harvest_effect"] = np.sin(2 * np.pi * (df["month"] - 3) / 12) + np.sin(
        2 * np.pi * (df["month"] - 9) / 12
    )

    # Modify the price_per_kg based on harvest effect
    df["price_per_kg"] = df["price_per_kg"] - df["harvest_effect"] * 0.5

    # Adjust promo periods to coincide with periods lagging peak harvest by 1 month
    peak_months = [4, 10]  # months following the peak availability
    df["promo"] = np.where(
        df["month"].isin(peak_months),
        1,
        np.random.choice([0, 1], n_rows, p=[0.85, 0.15]),
    )

    # Generate target variable based on features
    base_price_effect = -df["price_per_kg"] * 50
    seasonality_effect = df["harvest_effect"] * 50
    promo_effect = df["promo"] * 200

    df["demand"] = (
        base_demand
        + base_price_effect
        + seasonality_effect
        + promo_effect
        + df["weekend"] * 300
        + np.random.normal(0, 50, n_rows)
    ) * df[
        "inflation_multiplier"
    ]  # adding random noise

    # Add previous day's demand
    df["previous_days_demand"] = df["demand"].shift(1)
    # df["previous_days_demand"].fillna(
    #     method="bfill", inplace=True
    # )  # fill the first row

    df["previous_days_demand"] = df["previous_days_demand"].bfill()

    # Drop temporary columns
    df.drop(columns=["inflation_multiplier", "harvest_effect", "month"], inplace=True)

    return df

In [9]:
data = generate_apple_sales_data_with_promo_adjustment(base_demand=1_000, n_rows=1_000)

data[-20:]

,date,average_temperature,rainfall,weekend,holiday,price_per_kg,promo,demand,previous_days_demand
980,2025-09-05 11:03:44.317694,34.130183,1.454065,0,0,1.449177,0,999.306290,1029.418398
981,2025-09-06 11:03:44.317694,32.353643,9.462859,1,0,2.856503,0,1169.129427,999.306290
982,2025-09-07 11:03:44.317694,18.816833,0.391470,1,0,1.326429,0,1317.616709,1169.129427
983,2025-09-08 11:03:44.317694,34.533012,2.120477,0,0,0.970131,0,1068.802075,1317.616709
984,2025-09-09 11:03:44.317694,23.057202,2.365705,0,0,1.049931,0,1019.486305,1068.802075
985,2025-09-10 11:03:44.317694,34.810165,3.089005,0,0,2.035149,0,1002.564672,1019.486305
986,2025-09-11 11:03:44.317694,29.208905,3.673292,0,0,2.518098,0,1086.143402,1002.564672
987,2025-09-12 11:03:44.317694,16.428676,4.077782,0,0,1.268979,0,1093.207186,1086.143402
988,2025-09-13 11:03:44.317694,32.067512,2.734454,1,0,0.762317,0,1396.939894,1093.207186
989,2025-09-14 11:03:44.317694,31.938203,13.883486,1,0,1.153301,0,1321.409540,1396.939894


In [17]:
data.head()

,date,average_temperature,rainfall,weekend,holiday,price_per_kg,promo,demand,previous_days_demand
0,2022-12-29 17:42:52.082775,30.584727,6.786845,0,0,2.502464,0,844.994017,844.994017
1,2022-12-30 17:42:52.082775,15.465069,9.716520,0,0,1.871180,0,879.029699,844.994017
2,2022-12-31 17:42:52.082775,10.786525,1.099836,1,0,1.149160,0,1273.393597,879.029699
3,2023-01-01 17:42:52.082775,23.648154,9.578136,1,0,0.891414,0,1306.843795,1273.393597
4,2023-01-02 17:42:52.082775,13.861391,4.693826,0,0,0.737711,0,899.348986,1306.843795


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  1000 non-null   datetime64[ns]
 1   average_temperature   1000 non-null   float64       
 2   rainfall              1000 non-null   float64       
 3   weekend               1000 non-null   int64         
 4   holiday               1000 non-null   int64         
 5   price_per_kg          1000 non-null   float64       
 6   promo                 1000 non-null   int64         
 7   demand                1000 non-null   float64       
 8   previous_days_demand  1000 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(3)
memory usage: 70.4 KB


In [18]:
import pandas as pd
print(pd.__version__)

2.3.2


In [1]:
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [10]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")

In [11]:
# Sets the current active experiment to the "Apple_Models" experiment and
# returns the Experiment metadata
apple_experiment = mlflow.set_experiment("Apple_Models")

# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "apples_rf_test"

# Define an artifact path that the model will be saved to.
artifact_path = "rf_apples"

In [13]:
from mlflow import sklearn as mlflow_sklearn

In [14]:
# Split the data into features and target and drop irrelevant date field and target field
X = data.drop(columns=["date", "demand"])
y = data["demand"]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    "n_estimators": 100,
    "max_depth": 6,
    "min_samples_split": 10,
    "min_samples_leaf": 4,
    "bootstrap": True,
    "oob_score": False,
    "random_state": 888,
}

# Train the RandomForestRegressor
rf = RandomForestRegressor(**params)

# Fit the model on the training data
rf.fit(X_train, y_train)

# Predict on the validation set
y_pred = rf.predict(X_val)

# Calculate error metrics
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

# Assemble the metrics we're going to write into a collection
metrics = {"mae": mae, "mse": mse, "rmse": rmse, "r2": r2}

# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:
    # Log the parameters used for the model fit
    mlflow.log_params(params)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Log an instance of the trained model for later use
    mlflow.sklearn.log_model(sk_model=rf, input_example=X_val, name=artifact_path)

d:\MyProjects\mlflow_tutorial\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/09/24 11:12:04 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run apples_rf_test at: http://127.0.0.1:8080/#/experiments/107281342746158191/runs/77684e14003c4231b418a48fbe72af0e
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/107281342746158191
